In [ ]:
# create/activate venv (ypd_venv)

# create
# python -m venv ypd_venv
# 
# activate
# ypd_venv\Scripts\activate


In [8]:
# This is it. Does it all for processing data. 

import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# Dash and Plotly imports
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import hilbert
from fpdf import FPDF

# ----------------- SETUP DB -----------------
db_path = "pitch_analysis.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS pitch_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    participant_name TEXT,
    pitch_date TEXT,
    pitch_type TEXT,
    filename TEXT,
    mid_u_dev REAL,
    rel_u_dev REAL,
    frame1_u_dev REAL,
    frame2_u_dev REAL,
    frame3_u_dev REAL,
    frame4_u_dev REAL,
    frame5_u_dev REAL,
    frame6_u_dev REAL,
    frame7_u_dev REAL,
    frame8_u_dev REAL,
    frame9_u_dev REAL,
    frame10_u_dev REAL,
    mid_pronation REAL,
    rel_pronation REAL,
    frame1_pronation REAL,
    frame2_pronation REAL,
    frame3_pronation REAL,
    frame4_pronation REAL,
    frame5_pronation REAL,
    frame6_pronation REAL,
    frame7_pronation REAL,
    frame8_pronation REAL,
    frame9_pronation REAL,
    frame10_pronation REAL
)
""")
conn.commit()
conn.close()

# ----------------- USER INPUT (folder selection) -----------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")


# ----------------- LOAD C3D FILE PATHS -----------------
c3d_files = [os.path.join(selected_folder, file)
             for file in os.listdir(selected_folder)
             if file.lower().endswith('.c3d')]

if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")

# ----------------- IDENTIFY STATIC TRIAL -----------------
static_files = [f for f in c3d_files if "static" in f.lower()]
if not static_files:
    raise FileNotFoundError("No static trial found in the dataset.")

# ----------------- HELPER FUNCTIONS -----------------
def compute_static_baseline(static_file):
    """ Compute average wrist angles from a static trial. """
    c3d = ezc3d.c3d(static_file)
    points = c3d["data"]["points"]
    marker_labels = c3d["parameters"]["POINT"]["LABELS"]["value"]
    total_frames = points.shape[2]

    # Get marker indices
    markers = resolve_marker_indices(marker_labels)
    if not markers:
        raise ValueError(f"Markers not found in static trial: {static_file}")

    # Compute average angles across all frames in static trial
    ulnar_deviation_vals = []
    pronation_vals = []
    flexion_vals = []

    for frame in range(total_frames):
        ulnar_deviation_vals.append(compute_ulnar_deviation(points, markers, frame))
        pronation_vals.append(compute_pronation(points, markers, frame))
        flexion_vals.append(compute_wrist_flexion(points, markers, frame))

    # Return the mean angle from static trial
    return {
        "ulnar_dev": np.mean(ulnar_deviation_vals),
        "pronation": np.mean(pronation_vals),
        "flexion": np.mean(flexion_vals)
    }

def lowpass_filter(data, cutoff, fs, order=2):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=-1)

def compute_ulnar_deviation(points, marker_indices, frame):
    """ Computes ulnar deviation angle at a given frame. """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0

    forearm_vec = E_center - W_center
    radial_vec = R - U
    hand_vec = H - W_center

    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    radial_proj = radial_vec - np.dot(radial_vec, forearm_unit) * forearm_unit
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit) * forearm_unit

    if np.linalg.norm(radial_proj) < 1e-9 or np.linalg.norm(hand_proj) < 1e-9:
        return 0.0

    radial_proj_n = radial_proj / np.linalg.norm(radial_proj)
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)

    cross_val = np.cross(radial_proj_n, hand_proj_n)
    dot_val = np.dot(radial_proj_n, hand_proj_n)

    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    angle_deg = np.degrees(angle_rad) * sign
    return 180 - abs(angle_deg)

def compute_pronation(points, marker_indices, frame):
    """ Computes pronation/supination angle at a given frame. """
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]

    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0

    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)

    wrist_vec = R - U
    wrist_proj = wrist_vec - np.dot(wrist_vec, forearm_unit) * forearm_unit

    elbow_vec = E_lat - E_med
    elbow_proj = elbow_vec - np.dot(elbow_vec, forearm_unit) * forearm_unit

    if np.linalg.norm(wrist_proj) < 1e-9 or np.linalg.norm(elbow_proj) < 1e-9:
        return 0.0

    wrist_proj_n = wrist_proj / np.linalg.norm(wrist_proj)
    elbow_proj_n = elbow_proj / np.linalg.norm(elbow_proj)

    cross_val = np.cross(elbow_proj_n, wrist_proj_n)
    dot_val = np.dot(elbow_proj_n, wrist_proj_n)

    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))
    sign = np.sign(np.dot(cross_val, forearm_unit))

    raw_angle = np.degrees(angle_rad) * sign
    return 180 - abs(raw_angle)

def resolve_marker_indices(marker_labels):
    """ Map marker names to indices, removing left/right distinction. """
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]
    req = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    marker_indices = {}
    for mk in req:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}
    return marker_indices

# ----------------- APPLY STATIC TRIAL NORMALIZATION -----------------
static_baseline = compute_static_baseline(static_files[0])
print(f"Static Baseline: {static_baseline}")

# ----------------- APPLY TO PITCH DATA -----------------
normalized_data = []
for c3d_file in c3d_files:
    if "static" in c3d_file.lower():
        continue  # Skip static trials

    c3d_obj = ezc3d.c3d(c3d_file)
    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]

    markers = resolve_marker_indices(marker_labels)
    if not markers:
        continue

    # Compute angles and subtract static baseline
    for frame in range(points.shape[2]):
        ulnar_dev = compute_ulnar_deviation(points, markers, frame) - static_baseline["ulnar_dev"]
        pronation = compute_pronation(points, markers, frame) - static_baseline["pronation"]
        normalized_data.append((ulnar_dev, pronation))

print("Normalization complete.")
# ------------------------------------------------------------
#  MAIN PROCESSING LOOP: Insert results into pitch_data
# ------------------------------------------------------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for c3d_file_path in c3d_files:
    normalized_path = os.path.normpath(c3d_file_path)
    path_parts = normalized_path.split(os.sep)
    if len(path_parts) < 3:
        continue

    participant_folder = path_parts[-3]
    participant_name = participant_folder.rsplit("_", 1)[0]
    date_folder = path_parts[-2]
    pitch_date = date_folder.rstrip("_")
    filename_only = path_parts[-1]
    filename_noext = os.path.splitext(filename_only)[0]

    # pitch_type is the first word in the filename (split by space)
    pitch_type = filename_noext.split()[0].capitalize()

    try:
        c3d_obj = ezc3d.c3d(c3d_file_path)
    except Exception as e:
        print(f"Error reading {c3d_file_path}: {e}")
        continue

    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]

    try:
        foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
    except Exception as e:
        print(f"Skipping {c3d_file_path}: {e}")
        continue

    # ensure we have enough frames
    if release_frame + 20 >= total_frames:
        print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
        continue

    markers = resolve_marker_indices(marker_labels)
    if not markers:
        print(f"Skipping {c3d_file_path}: Required markers not found.")
        continue

    num_frames = release_frame - foot_contact_frame
    if num_frames < 1:
        print(f"Skipping {c3d_file_path}: release <= foot_contact.")
        continue

    mid_frame = foot_contact_frame + (num_frames // 2)

    # 11 equally spaced indexes from foot_contact to release
    equally_spaced = np.linspace(foot_contact_frame, release_frame, num=11, dtype=int)

    # Ulnar Deviation
    mid_u_dev = compute_ulnar_deviation(points, markers, mid_frame)
    rel_u_dev = compute_ulnar_deviation(points, markers, release_frame)
    frame_u_dev = []
    for i in range(1, 11):
        f_idx = equally_spaced[i]
        angle = compute_ulnar_deviation(points, markers, f_idx)
        frame_u_dev.append(angle)

    # Pronation
    mid_pronation = compute_pronation(points, markers, mid_frame)
    rel_pronation = compute_pronation(points, markers, release_frame)
    frame_pronation = []
    for i in range(1, 11):
        f_idx = equally_spaced[i]
        angle = compute_pronation(points, markers, f_idx)
        frame_pronation.append(angle)

    # Insert row into pitch_data
    insert_sql = """
    INSERT INTO pitch_data (
        participant_name, pitch_date, pitch_type, filename,
        mid_u_dev, rel_u_dev,
        frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
        frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
        mid_pronation, rel_pronation,
        frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
        frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
    )
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        participant_name,
        pitch_date,
        pitch_type,
        filename_noext,

        float(mid_u_dev),
        float(rel_u_dev),
        float(frame_u_dev[0]),
        float(frame_u_dev[1]),
        float(frame_u_dev[2]),
        float(frame_u_dev[3]),
        float(frame_u_dev[4]),
        float(frame_u_dev[5]),
        float(frame_u_dev[6]),
        float(frame_u_dev[7]),
        float(frame_u_dev[8]),
        float(frame_u_dev[9]),

        float(mid_pronation),
        float(rel_pronation),
        float(frame_pronation[0]),
        float(frame_pronation[1]),
        float(frame_pronation[2]),
        float(frame_pronation[3]),
        float(frame_pronation[4]),
        float(frame_pronation[5]),
        float(frame_pronation[6]),
        float(frame_pronation[7]),
        float(frame_pronation[8]),
        float(frame_pronation[9])
    )

    cursor.execute(insert_sql, data_tuple)
    print(f"Inserted data for {filename_noext} ({pitch_type})")

conn.commit()
conn.close()
print("All data inserted into pitch_data!")

# Retrieve the *last inserted* participant/date to set as defaults in Dash.
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT participant_name, pitch_date FROM pitch_data ORDER BY id DESC LIMIT 1")
row = c.fetchone()
conn.close()

if row:
    LAST_PARTICIPANT, LAST_DATE = row
else:
    LAST_PARTICIPANT, LAST_DATE = (None, None)

# ----------------- GET TIME SERIES (for graphs) -----------------
def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    ts = {}
    for c3d_file_path in c3d_files:
        normalized_path = os.path.normpath(c3d_file_path)
        path_parts = normalized_path.split(os.sep)
        if len(path_parts) < 3:
            continue
        participant_folder = path_parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder = path_parts[-2].rstrip("_")

        if participant_name != selected_participant or date_folder != selected_date:
            continue

        filename_only = path_parts[-1]
        filename_noext = os.path.splitext(filename_only)[0]
        pitch_type = filename_noext.split()[0].capitalize()

        if selected_pitch_type != "All" and pitch_type != selected_pitch_type:
            continue
        if selected_filename != "All" and filename_noext != selected_filename:
            continue

        try:
            c3d_obj = ezc3d.c3d(c3d_file_path)
        except Exception as e:
            print(f"Error reading {c3d_file_path}: {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_file_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame = release_frame + 20
        ulnar_series = []
        pronation_series = []
        flexion_series = []

        # Collect angles from foot_contact..release+20
        for frame in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame)
            p_angle = compute_pronation(points, markers, frame)
            f_angle = compute_wrist_flexion(points, markers, frame)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        # Convert to numpy
        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        # # Subtract baseline so it starts near 0 (optional). 
        # # This can cause a mismatch vs. reference data if the reference isn't also zeroed.
        # pronation_series = pronation_series - pronation_series[0]
        # flexion_series = flexion_series - flexion_series[0]

        if pitch_type not in ts:
            ts[pitch_type] = {"ulnar_dev_series": [], "pronation": [], "flexion": []}

        ts[pitch_type]["ulnar_dev_series"].append(ulnar_series)
        ts[pitch_type]["pronation"].append(pronation_series)
        ts[pitch_type]["flexion"].append(flexion_series)

    return ts

def get_curve_reference_on_the_fly():
    """
    Only loads 'Curve' .c3d files from a known reference folder and averages them.
    Adjust as needed for your actual reference approach.
    """
    REFERENCE_FOLDER = r"D:\Youth Pitch Design\Data\Reference Data_RD\2025-02-21_"
    if not os.path.isdir(REFERENCE_FOLDER):
        return {}

    all_files = [
        os.path.join(REFERENCE_FOLDER, f)
        for f in os.listdir(REFERENCE_FOLDER)
        if f.lower().endswith('.c3d') and "curve" in f.lower()
    ]
    if not all_files:
        print("No valid 'Curve' .c3d files found in reference folder.")
        return {}

    all_ulnar_arrays = []
    all_pronation_arrays = []
    all_flexion_arrays = []

    for c3d_path in all_files:
        try:
            c3d_obj = ezc3d.c3d(c3d_path)
        except Exception as e:
            print(f"Skipping {c3d_path}: read error {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame   = release_frame + 20

        ulnar_series = []
        pronation_series = []
        flexion_series = []
        for frame_idx in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame_idx)
            p_angle = compute_pronation(points, markers, frame_idx)
            f_angle = compute_wrist_flexion(points, markers, frame_idx)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        all_ulnar_arrays.append(ulnar_series)
        all_pronation_arrays.append(pronation_series)
        all_flexion_arrays.append(flexion_series)

    if not all_ulnar_arrays:
        return {}

    # Trim to min length across all files
    min_len = min([arr.shape[0] for arr in all_ulnar_arrays + all_pronation_arrays + all_flexion_arrays])
    all_ulnar_arrays = [arr[:min_len] for arr in all_ulnar_arrays]
    all_pronation_arrays = [arr[:min_len] for arr in all_pronation_arrays]
    all_flexion_arrays = [arr[:min_len] for arr in all_flexion_arrays]

    stacked_ulnar = np.vstack(all_ulnar_arrays)
    stacked_pron  = np.vstack(all_pronation_arrays)
    stacked_flex  = np.vstack(all_flexion_arrays)

    mean_ulnar = np.mean(stacked_ulnar, axis=0)
    mean_pron  = np.mean(stacked_pron,  axis=0)
    mean_flex  = np.mean(stacked_flex,  axis=0)

    # Return only "Curve" references
    ref_data = {
        "Curve": {
            "ulnar_dev_series": [mean_ulnar],
            "pronation": [mean_pron],
            "flexion": [mean_flex]
        }
    }
    return ref_data

# For a table-based reference (if you have a 'reference_data' table), you'd have a function like:
# def get_reference_time_series():
#     ... your table approach ...
#     return reference_ts

# ----------------- HELPER FUNCTIONS -----------------
def get_dropdown_options():
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_data", conn)
    conn.close()
    participants = df['participant_name'].unique()
    options = [{"label": p, "value": p} for p in sorted(participants)]
    return options

def get_date_options(selected_participant):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_date FROM pitch_data WHERE participant_name = '{selected_participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique()
    return [{"label": d, "value": d} for d in sorted(dates)]

def get_pitch_type_options(selected_participant, selected_date):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_type FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": pt, "value": pt} for pt in sorted(df["pitch_type"].unique())]
    # Insert "All" at the top
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_filename_options(selected_participant, selected_date, selected_pitch_type):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT filename FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": fn, "value": fn} for fn in sorted(df["filename"].unique())]
    # Insert "All"
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_comparison_table(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """Shows a table that includes both Ulnar Deviation and Pronation comparisons."""
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"
    pitch_df = pd.read_sql_query(query, conn)

    # If you have a reference_data table, fetch it here:
    ref_df = pd.read_sql_query("SELECT * FROM reference_data", conn)
    conn.close()

    # Group for selected data
    pitch_summary = pitch_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    # Group for reference data
    ref_summary = ref_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    merged = pitch_summary.merge(ref_summary, on="pitch_type", suffixes=("_selected", "_reference"))

    # Simple "acceleration" estimate from frame2..frame10
    # For Ulnar Dev
    merged["accel_u_dev_selected"] = (merged["frame10_u_dev_selected"] - merged["frame2_u_dev_selected"]) / 8.0
    merged["accel_u_dev_reference"] = (merged["frame10_u_dev_reference"] - merged["frame2_u_dev_reference"]) / 8.0
    merged["diff_accel_u_dev"] = merged["accel_u_dev_selected"] - merged["accel_u_dev_reference"]

    # For Pronation
    merged["accel_pronation_selected"] = (merged["frame10_pronation_selected"] - merged["frame2_pronation_selected"]) / 8.0
    merged["accel_pronation_reference"] = (merged["frame10_pronation_reference"] - merged["frame2_pronation_reference"]) / 8.0
    merged["diff_accel_pronation"] = merged["accel_pronation_selected"] - merged["accel_pronation_reference"]

    rows = []
    for _, row in merged.iterrows():
        pitch = row["pitch_type"]
        # Ulnar Deviation row
        rows.append({
            "pitch_type": pitch,
            "rel_u_dev": round(row["rel_u_dev_selected"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"], 1),
            "accel_u_dev": round(row["accel_u_dev_selected"], 1)
        })
        # Ulnar Deviation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Comp",
            "rel_u_dev": round(row["rel_u_dev_selected"] - row["rel_u_dev_reference"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"] - row["frame2_u_dev_reference"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"] - row["frame4_u_dev_reference"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"] - row["frame6_u_dev_reference"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"] - row["frame8_u_dev_reference"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"] - row["frame10_u_dev_reference"], 1),
            "accel_u_dev": round(row["diff_accel_u_dev"], 1)
        })
        # Pronation row (store actual pronation at release in rel_u_dev column)
        rows.append({
            "pitch_type": f"{pitch} Pronation",
            # Instead of "Pronation", place the numeric pronation at release:
            "rel_u_dev": round(row["rel_pronation_selected"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"], 1),
            "accel_u_dev": round(row["accel_pronation_selected"], 1)
        })
        # Pronation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Pronation Comp",
            "rel_u_dev": round(row["rel_pronation_selected"] - row["rel_pronation_reference"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"] - row["frame2_pronation_reference"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"] - row["frame4_pronation_reference"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"] - row["frame6_pronation_reference"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"] - row["frame8_pronation_reference"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"] - row["frame10_pronation_reference"], 1),
            "accel_u_dev": round(row["diff_accel_pronation"], 1)
        })

    comp_df = pd.DataFrame(rows)
    return comp_df


# ----------------- DASH APP SETUP -----------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.title = "Pitch Analysis Dashboard"

logo_url = "https://8ctanebaseball.com/wp-content/uploads/2024/02/cropped-8ctaneBaseballLogo-2.png"

# We define global defaults from the last inserted row:
default_participant = LAST_PARTICIPANT
default_date = LAST_DATE

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label("Select Participant", style={"color": "white"}),
            dcc.Dropdown(
                id="participant-dropdown",
                options=get_dropdown_options(),
                value=default_participant,  # Use last processed participant
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Test Date", style={"color": "white"}),
            dcc.Dropdown(
                id="date-dropdown",
                # We leave options empty; the callback will populate them
                options=[],
                value=default_date,  # Use last processed date
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Type", style={"color": "white"}),
            dcc.Dropdown(
                id="pitch-type-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Number", style={"color": "white"}),
            dcc.Dropdown(
                id="filename-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Div([
                html.Img(
                    src=logo_url,
                    style={
                        "position": "relative", 
                        "width": "440px",
                        "height": "auto",
                        "padding": "10px",  
                        "margin-left": "140px",
                        "backgroundColor": "black"  
                    }
                )
            ], style={"text-align": "right"})
        ], width=2)
    ], className="mt-3", align="center"),       

    html.Hr(),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Comparison Table"),
                dbc.CardBody(
                    dash_table.DataTable(
                        id="comparison-table",
                        columns=[{"name": i, "id": i} for i in [
                            "pitch_type", "rel_u_dev",
                            "frame2_u_dev", "frame4_u_dev",
                            "frame6_u_dev", "frame8_u_dev",
                            "frame10_u_dev", "accel_u_dev"
                        ]],
                        data=[],
                        style_table={"overflowX": "auto"},
                        style_cell={
                            "textAlign": "center",
                            "color": "white",
                            "backgroundColor": "black"
                        },
                        style_header={
                            "backgroundColor": "#333333",
                            "color": "white"
                        },
                        page_size=10
                    )
                )
            ]),
            width=12
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Ulnar Deviation Time Series"),
                dbc.CardBody([
                    dcc.Graph(id="ulnar-dev-graph"),
                    html.P([
                        "Ulnar deviation measures how much the wrist is angled toward the ulna side or 'flicks' as we release the ball.",
                        html.Br(),
                        html.Strong("Moving in the negative (-) direction represents ulnar deviation "),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
                ])
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Acceleration: Transverse & Frontal"),
                dbc.CardBody([
                    dcc.Graph(id="acceleration-graph"),
                    html.P([
                        "Acceleration in the transverse and frontal planes shows how "
                            "rapidly the wrist angles are changing around release, "
                            "which can correlate with injury risk. ",
                        html.Br(),
                        html.Strong("Ideally, it’s kept minimal "),
                        "to reduce stress during throwing."
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
                ])
            ]), width=6
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Pronation Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="pronation-graph")),
                    html.P([
                        "Pronation and Supination measure how much you 'twist' the wrist. Supination through ball release is associated with the same 'flick' motion we are trying to avoid",
                        html.Br(),
                        html.Strong("Negative (-) values correspond with supination"),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Flexion Time Series"),
                dbc.CardBody(
                    dcc.Graph(id="flexion-graph")),
                    html.P([
                        "Flexion at the wrist can be associated with the same 'flick' that occurs with excessive ulnar deviation",
                        html.Br(),
                        html.Strong("Negative (-) values correspond with flexion"),
                    ], style={
                        "marginTop": "10px", 
                        "fontSize": "22px", 
                        "lineHeight": "2.4"
                    })
            ]), width=6
        )
    ], className="mt-3")
], fluid=True)


# ----------------- DASH CALLBACKS -----------------
@app.callback(
    Output("date-dropdown", "options"),
    Output("date-dropdown", "value"),
    Input("participant-dropdown", "value")
)
def update_date_dropdown(selected_participant):
    if not selected_participant:
        return [], None
    options = get_date_options(selected_participant)
    # If the default_date is in the new options, keep it, else use the first
    possible_values = [o["value"] for o in options]
    if default_date in possible_values:
        return options, default_date
    return options, (options[0]["value"] if options else None)

@app.callback(
    Output("pitch-type-dropdown", "options"),
    Output("pitch-type-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_pitch_type_options(selected_participant, selected_date):
    if not selected_participant or not selected_date:
        return [], None
    options = get_pitch_type_options(selected_participant, selected_date)
    pitch_types = [o["value"] for o in options]

    # Default to "Curve" if available, else "All" or the first
    if "Curve" in pitch_types:
        default_value = "Curve"
    else:
        default_value = "All" if "All" in pitch_types else (pitch_types[0] if pitch_types else None)

    return options, default_value

@app.callback(
    Output("filename-dropdown", "options"),
    Output("filename-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("pitch-type-dropdown", "value")
)
def update_filename_options(selected_participant, selected_date, selected_pitch_type):
    if not selected_participant or not selected_date or not selected_pitch_type:
        return [], None
    options = get_filename_options(selected_participant, selected_date, selected_pitch_type)
    value = options[0]["value"] if options else None
    return options, value

# ... your existing imports and code

@app.callback(
    Output("comparison-table", "data"),
    Output("ulnar-dev-graph", "figure"),
    Output("acceleration-graph", "figure"),
    Output("pronation-graph", "figure"),
    Output("flexion-graph", "figure"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value"),
        Input("filename-dropdown", "value")
    ]
)
def update_dashboard(selected_participant, selected_date, selected_pitch_type, selected_filename):
    # 1) Table & 2) main time-series
    comp_df = get_comparison_table(selected_participant, selected_date, selected_pitch_type, selected_filename)
    ts_data = get_time_series(selected_participant, selected_date, selected_pitch_type, selected_filename)
    table_data = comp_df.to_dict("records")

    # 3) reference data for "Curve"
    ref_data = get_curve_reference_on_the_fly()

    # Compute average release index
    release_frames = []
    for pt, data_dict in ts_data.items():
        for series in data_dict["ulnar_dev_series"]:
            # The last 20 frames in each series are post-release
            release_idx = len(series) - 20
            release_frames.append(release_idx)
    avg_release = int(np.mean(release_frames)) if release_frames else 20

    vline = dict(
        type="line",
        x0=avg_release, x1=avg_release,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    pitch_color_map = {
        "Fastball": "#d79ea5",
        "Curve": "#2c99d4",
        "Slider": "#ff9900",
        "Changeup": "#ffff00"
    }

    # --------------- Ulnar Deviation ---------------
    ulnar_fig = go.Figure()
    plotted_pitch_types = set()  # Track pitch types that have been added to the legend

    # Plot reference if relevant
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    # Plot actual data
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Show legend only once per pitch type
    
        for series in data_dict["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,  # Only show legend entry for the first trace
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)

    ulnar_fig.update_layout(
        title="Ulnar Deviation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline],
        autosize=True  # Let it expand
    )

    # --------------- Acceleration ---------------
    accel_fig = go.Figure()

    def compute_acceleration(angle_series):
        return np.diff(angle_series, n=2)

    def compute_rms(signal, window_size=5):
        squared = np.square(signal)
        kernel = np.ones(window_size) / window_size
        mean_sq = np.convolve(squared, kernel, mode='same')
        return np.sqrt(mean_sq)

    # We'll do 30 frames before release, 40 frames after
    FRAMES_BEFORE = 30
    FRAMES_AFTER  = 40
    
    # The release point is at the 30th index in the new zoomed-in window
    release_line = dict(
        type="line",
        x0=30, x1=30,  # Middle of the new range
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    plotted_pitch_types = set()
    
    for pt, data_dict in ts_data.items():
        color_ulnar = "#bb6a74"
        color_pron  = "#2c99d4"
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["ulnar_dev_series"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i   = min(release_index + FRAMES_AFTER, len(series))
            sub_series = series[start_i:end_i]
    
            acc_u = compute_acceleration(sub_series)
            rms_u = compute_rms(acc_u)
            x_vals = list(range(len(rms_u)))
    
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_u,
                mode="lines",
                line=dict(color=color_ulnar, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        for series in data_dict["pronation"]:
            release_index = len(series) - 20
            start_i = max(release_index - FRAMES_BEFORE, 0)
            end_i   = min(release_index + FRAMES_AFTER, len(series))
            sub_series = series[start_i:end_i]
    
            acc_p = compute_acceleration(sub_series)
            rms_p = compute_rms(acc_p)
            x_vals = list(range(len(rms_p)))
    
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_p,
                mode="lines",
                line=dict(color=color_pron, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    
    accel_fig.update_layout(
        title="Acceleration: ±30 Frames Before to +40 After Release",
        xaxis_title="Index (local to sub-window)",
        yaxis_title="Accel (°/frame²)",
        shapes=[release_line],
        autosize=True
    )
    
    # --------------- Pronation ---------------
    pronation_fig = go.Figure()
    plotted_pitch_types = set()
    
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    
    pronation_fig.update_layout(
        title="Pronation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )
    
    # --------------- Flexion ---------------
    flexion_fig = go.Figure()
    plotted_pitch_types = set()
    
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        show_legend = pt not in plotted_pitch_types  # Only show first legend entry
    
        for series in data_dict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt if show_legend else None,
                showlegend=show_legend
            ))
    
        plotted_pitch_types.add(pt)
    
    flexion_fig.update_layout(
        title="Flexion Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    return table_data, ulnar_fig, accel_fig, pronation_fig, flexion_fig


if __name__ == '__main__':
    app.run_server(debug=True)


Static Baseline: {'ulnar_dev': np.float64(1.3282930737937069), 'pronation': np.float64(101.26739594799018), 'flexion': np.float64(28.335479663584888)}
Normalization complete.
Inserted data for Curve RH 5 (Curve)
Inserted data for Curve RH 4 (Curve)
Inserted data for Curve RH 3 (Curve)
Inserted data for Curve RH 1 (Curve)
Inserted data for Fastball RH 2 (Fastball)
Inserted data for Fastball RH 1 (Fastball)
Skipping D:/Youth Pitch Design/Data/Caleb Sasser_CS/2025-02-13_\Pitch Design Static RH 1.c3d: No EVENT data in C3D.
Inserted data for Curve RH 2 (Curve)
All data inserted into pitch_data!


In [59]:
import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

# Dash and Plotly imports
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.io as pio
from scipy.signal import hilbert
from fpdf import FPDF

# ----------------- SETUP DB -----------------
db_path = "pitch_analysis.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS pitch_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    participant_name TEXT,
    pitch_date TEXT,
    pitch_type TEXT,
    filename TEXT,
    mid_u_dev REAL,
    rel_u_dev REAL,
    frame1_u_dev REAL,
    frame2_u_dev REAL,
    frame3_u_dev REAL,
    frame4_u_dev REAL,
    frame5_u_dev REAL,
    frame6_u_dev REAL,
    frame7_u_dev REAL,
    frame8_u_dev REAL,
    frame9_u_dev REAL,
    frame10_u_dev REAL,
    mid_pronation REAL,
    rel_pronation REAL,
    frame1_pronation REAL,
    frame2_pronation REAL,
    frame3_pronation REAL,
    frame4_pronation REAL,
    frame5_pronation REAL,
    frame6_pronation REAL,
    frame7_pronation REAL,
    frame8_pronation REAL,
    frame9_pronation REAL,
    frame10_pronation REAL
)
""")
conn.commit()
conn.close()

# ----------------- USER INPUT (folder selection) -----------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")

# ----------------- LOAD C3D FILE PATHS -----------------
c3d_files = [os.path.join(selected_folder, file)
             for file in os.listdir(selected_folder)
             if file.lower().endswith('.c3d')]
if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")

# ----------------- COMPUTE FUNCTIONS -----------------
def lowpass_filter(data, cutoff, fs, order=2):
    nyquist = 0.5 * fs
    normal_cutoff = cutoff / nyquist
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return filtfilt(b, a, data, axis=-1)

def compute_ulnar_deviation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]
    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = E_center - W_center
    radial_vec  = R - U
    hand_vec    = H - W_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    radial_proj = radial_vec - np.dot(radial_vec, forearm_unit)*forearm_unit
    hand_proj   = hand_vec   - np.dot(hand_vec,   forearm_unit)*forearm_unit
    if np.linalg.norm(radial_proj) < 1e-9 or np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    radial_proj_n = radial_proj / np.linalg.norm(radial_proj)
    hand_proj_n   = hand_proj   / np.linalg.norm(hand_proj)
    cross_val = np.cross(radial_proj_n, hand_proj_n)
    dot_val   = np.dot(radial_proj_n, hand_proj_n)
    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    angle_deg = np.degrees(angle_rad) * sign
    return 180 - abs(angle_deg)

def compute_pronation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    wrist_vec = R - U
    wrist_proj = wrist_vec - np.dot(wrist_vec, forearm_unit)*forearm_unit
    elbow_vec = E_lat - E_med
    elbow_proj = elbow_vec - np.dot(elbow_vec, forearm_unit)*forearm_unit
    if np.linalg.norm(wrist_proj) < 1e-9 or np.linalg.norm(elbow_proj) < 1e-9:
        return 0.0
    wrist_proj_n = wrist_proj / np.linalg.norm(wrist_proj)
    elbow_proj_n = elbow_proj / np.linalg.norm(elbow_proj)
    cross_val = np.cross(elbow_proj_n, wrist_proj_n)
    dot_val = np.dot(elbow_proj_n, wrist_proj_n)
    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))
    sign = np.sign(np.dot(cross_val, forearm_unit))
    raw_angle = np.degrees(angle_rad) * sign
    return 180 - abs(raw_angle)

def compute_wrist_flexion(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]
    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    vertical = np.array([0, 0, 1])
    neutral = vertical - np.dot(vertical, forearm_unit)*forearm_unit
    if np.linalg.norm(neutral) < 1e-9:
        return 0.0
    neutral_n = neutral / np.linalg.norm(neutral)
    hand_vec = H - W_center
    hand_proj = hand_vec - np.dot(hand_vec, forearm_unit)*forearm_unit
    if np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    hand_proj_n = hand_proj / np.linalg.norm(hand_proj)
    dot_val = np.dot(neutral_n, hand_proj_n)
    dot_val = np.clip(dot_val, -1.0, 1.0)
    angle_rad = np.arccos(dot_val)
    cross_val = np.cross(neutral_n, hand_proj_n)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    raw_angle = np.degrees(angle_rad) * sign
    return 180 - abs(raw_angle)

def find_local_events(c3d, frame_rate, total_frames):
    if "EVENT" not in c3d["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]
    if "Foot Contact" not in event_labels or "Release" not in event_labels:
        raise ValueError("Required events (Foot Contact, Release) not found in C3D events.")
    idx_foot = event_labels.index("Foot Contact")
    idx_release = event_labels.index("Release")
    foot_contact_sec = event_times[idx_foot]
    release_sec = event_times[idx_release]
    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global = int(round(release_sec * frame_rate))
    earliest = min(foot_contact_global, release_global)
    foot_local = foot_contact_global - earliest
    release_local = release_global - earliest
    if foot_local < 0 or foot_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")
    return foot_local, release_local

def resolve_marker_indices(marker_labels):
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]
    req = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    marker_indices = {}
    for mk in req:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}
    return marker_indices

# ------------------------------------------------------------
#  MAIN PROCESSING LOOP: Insert results into pitch_data
# ------------------------------------------------------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for c3d_file_path in c3d_files:
    normalized_path = os.path.normpath(c3d_file_path)
    path_parts = normalized_path.split(os.sep)
    if len(path_parts) < 3:
        continue

    participant_folder = path_parts[-3]
    participant_name = participant_folder.rsplit("_", 1)[0]
    date_folder = path_parts[-2]
    pitch_date = date_folder.rstrip("_")
    filename_only = path_parts[-1]
    filename_noext = os.path.splitext(filename_only)[0]

    # pitch_type is the first word in the filename (split by space)
    pitch_type = filename_noext.split()[0].capitalize()

    try:
        c3d_obj = ezc3d.c3d(c3d_file_path)
    except Exception as e:
        print(f"Error reading {c3d_file_path}: {e}")
        continue

    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]

    try:
        foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
    except Exception as e:
        print(f"Skipping {c3d_file_path}: {e}")
        continue

    # ensure we have enough frames
    if release_frame + 20 >= total_frames:
        print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
        continue

    markers = resolve_marker_indices(marker_labels)
    if not markers:
        print(f"Skipping {c3d_file_path}: Required markers not found.")
        continue

    num_frames = release_frame - foot_contact_frame
    if num_frames < 1:
        print(f"Skipping {c3d_file_path}: release <= foot_contact.")
        continue

    mid_frame = foot_contact_frame + (num_frames // 2)

    # 11 equally spaced indexes from foot_contact to release
    equally_spaced = np.linspace(foot_contact_frame, release_frame, num=11, dtype=int)

    # Ulnar Deviation
    mid_u_dev = compute_ulnar_deviation(points, markers, mid_frame)
    rel_u_dev = compute_ulnar_deviation(points, markers, release_frame)
    frame_u_dev = []
    for i in range(1, 11):
        f_idx = equally_spaced[i]
        angle = compute_ulnar_deviation(points, markers, f_idx)
        frame_u_dev.append(angle)

    # Pronation
    mid_pronation = compute_pronation(points, markers, mid_frame)
    rel_pronation = compute_pronation(points, markers, release_frame)
    frame_pronation = []
    for i in range(1, 11):
        f_idx = equally_spaced[i]
        angle = compute_pronation(points, markers, f_idx)
        frame_pronation.append(angle)

    # Insert row into pitch_data
    insert_sql = """
    INSERT INTO pitch_data (
        participant_name, pitch_date, pitch_type, filename,
        mid_u_dev, rel_u_dev,
        frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
        frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
        mid_pronation, rel_pronation,
        frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
        frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
    )
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        participant_name,
        pitch_date,
        pitch_type,
        filename_noext,

        float(mid_u_dev),
        float(rel_u_dev),
        float(frame_u_dev[0]),
        float(frame_u_dev[1]),
        float(frame_u_dev[2]),
        float(frame_u_dev[3]),
        float(frame_u_dev[4]),
        float(frame_u_dev[5]),
        float(frame_u_dev[6]),
        float(frame_u_dev[7]),
        float(frame_u_dev[8]),
        float(frame_u_dev[9]),

        float(mid_pronation),
        float(rel_pronation),
        float(frame_pronation[0]),
        float(frame_pronation[1]),
        float(frame_pronation[2]),
        float(frame_pronation[3]),
        float(frame_pronation[4]),
        float(frame_pronation[5]),
        float(frame_pronation[6]),
        float(frame_pronation[7]),
        float(frame_pronation[8]),
        float(frame_pronation[9])
    )

    cursor.execute(insert_sql, data_tuple)
    print(f"Inserted data for {filename_noext} ({pitch_type})")

conn.commit()
conn.close()
print("All data inserted into pitch_data!")

# Retrieve the *last inserted* participant/date to set as defaults in Dash.
conn = sqlite3.connect(db_path)
c = conn.cursor()
c.execute("SELECT participant_name, pitch_date FROM pitch_data ORDER BY id DESC LIMIT 1")
row = c.fetchone()
conn.close()

if row:
    LAST_PARTICIPANT, LAST_DATE = row
else:
    LAST_PARTICIPANT, LAST_DATE = (None, None)

# ----------------- GET TIME SERIES (for graphs) -----------------
def get_time_series(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    ts = {}
    for c3d_file_path in c3d_files:
        normalized_path = os.path.normpath(c3d_file_path)
        path_parts = normalized_path.split(os.sep)
        if len(path_parts) < 3:
            continue
        participant_folder = path_parts[-3]
        participant_name = participant_folder.rsplit("_", 1)[0]
        date_folder = path_parts[-2].rstrip("_")

        if participant_name != selected_participant or date_folder != selected_date:
            continue

        filename_only = path_parts[-1]
        filename_noext = os.path.splitext(filename_only)[0]
        pitch_type = filename_noext.split()[0].capitalize()

        if selected_pitch_type != "All" and pitch_type != selected_pitch_type:
            continue
        if selected_filename != "All" and filename_noext != selected_filename:
            continue

        try:
            c3d_obj = ezc3d.c3d(c3d_file_path)
        except Exception as e:
            print(f"Error reading {c3d_file_path}: {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_file_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_file_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_file_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame = release_frame + 20
        ulnar_series = []
        pronation_series = []
        flexion_series = []

        # Collect angles from foot_contact..release+20
        for frame in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame)
            p_angle = compute_pronation(points, markers, frame)
            f_angle = compute_wrist_flexion(points, markers, frame)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        # Convert to numpy
        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        # # Subtract baseline so it starts near 0 (optional). 
        # # This can cause a mismatch vs. reference data if the reference isn't also zeroed.
        # pronation_series = pronation_series - pronation_series[0]
        # flexion_series = flexion_series - flexion_series[0]

        if pitch_type not in ts:
            ts[pitch_type] = {"ulnar_dev_series": [], "pronation": [], "flexion": []}

        ts[pitch_type]["ulnar_dev_series"].append(ulnar_series)
        ts[pitch_type]["pronation"].append(pronation_series)
        ts[pitch_type]["flexion"].append(flexion_series)

    return ts

def get_curve_reference_on_the_fly():
    """
    Only loads 'Curve' .c3d files from a known reference folder and averages them.
    Adjust as needed for your actual reference approach.
    """
    REFERENCE_FOLDER = r"D:\Youth Pitch Design\Data\Reference Data_RD\2025-02-21_"
    if not os.path.isdir(REFERENCE_FOLDER):
        return {}

    all_files = [
        os.path.join(REFERENCE_FOLDER, f)
        for f in os.listdir(REFERENCE_FOLDER)
        if f.lower().endswith('.c3d') and "curve" in f.lower()
    ]
    if not all_files:
        print("No valid 'Curve' .c3d files found in reference folder.")
        return {}

    all_ulnar_arrays = []
    all_pronation_arrays = []
    all_flexion_arrays = []

    for c3d_path in all_files:
        try:
            c3d_obj = ezc3d.c3d(c3d_path)
        except Exception as e:
            print(f"Skipping {c3d_path}: read error {e}")
            continue

        points = c3d_obj["data"]["points"]
        marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
        frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
        total_frames = points.shape[2]

        try:
            foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
        except Exception as e:
            print(f"Skipping {c3d_path}: {e}")
            continue

        if release_frame + 20 >= total_frames:
            print(f"Skipping {c3d_path}: Not enough frames for release+20.")
            continue

        markers = resolve_marker_indices(marker_labels)
        if not markers:
            print(f"Skipping {c3d_path}: Required markers not found.")
            continue

        start_frame = foot_contact_frame
        end_frame   = release_frame + 20

        ulnar_series = []
        pronation_series = []
        flexion_series = []
        for frame_idx in range(start_frame, end_frame):
            u_angle = compute_ulnar_deviation(points, markers, frame_idx)
            p_angle = compute_pronation(points, markers, frame_idx)
            f_angle = compute_wrist_flexion(points, markers, frame_idx)
            ulnar_series.append(u_angle)
            pronation_series.append(p_angle)
            flexion_series.append(f_angle)

        ulnar_series = np.array(ulnar_series)
        pronation_series = np.array(pronation_series)
        flexion_series = np.array(flexion_series)

        all_ulnar_arrays.append(ulnar_series)
        all_pronation_arrays.append(pronation_series)
        all_flexion_arrays.append(flexion_series)

    if not all_ulnar_arrays:
        return {}

    # Trim to min length across all files
    min_len = min([arr.shape[0] for arr in all_ulnar_arrays + all_pronation_arrays + all_flexion_arrays])
    all_ulnar_arrays = [arr[:min_len] for arr in all_ulnar_arrays]
    all_pronation_arrays = [arr[:min_len] for arr in all_pronation_arrays]
    all_flexion_arrays = [arr[:min_len] for arr in all_flexion_arrays]

    stacked_ulnar = np.vstack(all_ulnar_arrays)
    stacked_pron  = np.vstack(all_pronation_arrays)
    stacked_flex  = np.vstack(all_flexion_arrays)

    mean_ulnar = np.mean(stacked_ulnar, axis=0)
    mean_pron  = np.mean(stacked_pron,  axis=0)
    mean_flex  = np.mean(stacked_flex,  axis=0)

    # Return only "Curve" references
    ref_data = {
        "Curve": {
            "ulnar_dev_series": [mean_ulnar],
            "pronation": [mean_pron],
            "flexion": [mean_flex]
        }
    }
    return ref_data

# For a table-based reference (if you have a 'reference_data' table), you'd have a function like:
# def get_reference_time_series():
#     ... your table approach ...
#     return reference_ts

# ----------------- HELPER FUNCTIONS -----------------
def get_dropdown_options():
    conn = sqlite3.connect(db_path)
    df = pd.read_sql_query("SELECT DISTINCT participant_name FROM pitch_data", conn)
    conn.close()
    participants = df['participant_name'].unique()
    options = [{"label": p, "value": p} for p in sorted(participants)]
    return options

def get_date_options(selected_participant):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_date FROM pitch_data WHERE participant_name = '{selected_participant}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    dates = df["pitch_date"].unique()
    return [{"label": d, "value": d} for d in sorted(dates)]

def get_pitch_type_options(selected_participant, selected_date):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT pitch_type FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": pt, "value": pt} for pt in sorted(df["pitch_type"].unique())]
    # Insert "All" at the top
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_filename_options(selected_participant, selected_date, selected_pitch_type):
    conn = sqlite3.connect(db_path)
    query = f"SELECT DISTINCT filename FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    df = pd.read_sql_query(query, conn)
    conn.close()
    options = [{"label": fn, "value": fn} for fn in sorted(df["filename"].unique())]
    # Insert "All"
    options.insert(0, {"label": "All", "value": "All"})
    return options

def get_comparison_table(selected_participant, selected_date, selected_pitch_type="All", selected_filename="All"):
    """Shows a table that includes both Ulnar Deviation and Pronation comparisons."""
    conn = sqlite3.connect(db_path)
    query = f"SELECT * FROM pitch_data WHERE participant_name = '{selected_participant}' AND pitch_date = '{selected_date}'"
    if selected_pitch_type != "All":
        query += f" AND pitch_type = '{selected_pitch_type}'"
    if selected_filename != "All":
        query += f" AND filename = '{selected_filename}'"
    pitch_df = pd.read_sql_query(query, conn)

    # If you have a reference_data table, fetch it here:
    ref_df = pd.read_sql_query("SELECT * FROM reference_data", conn)
    conn.close()

    # Group for selected data
    pitch_summary = pitch_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    # Group for reference data
    ref_summary = ref_df.groupby("pitch_type").agg({
        "rel_u_dev": "mean",
        "frame2_u_dev": "mean",
        "frame4_u_dev": "mean",
        "frame6_u_dev": "mean",
        "frame8_u_dev": "mean",
        "frame10_u_dev": "mean",
        "rel_pronation": "mean",
        "frame2_pronation": "mean",
        "frame4_pronation": "mean",
        "frame6_pronation": "mean",
        "frame8_pronation": "mean",
        "frame10_pronation": "mean"
    }).reset_index()

    merged = pitch_summary.merge(ref_summary, on="pitch_type", suffixes=("_selected", "_reference"))

    # Simple "acceleration" estimate from frame2..frame10
    # For Ulnar Dev
    merged["accel_u_dev_selected"] = (merged["frame10_u_dev_selected"] - merged["frame2_u_dev_selected"]) / 8.0
    merged["accel_u_dev_reference"] = (merged["frame10_u_dev_reference"] - merged["frame2_u_dev_reference"]) / 8.0
    merged["diff_accel_u_dev"] = merged["accel_u_dev_selected"] - merged["accel_u_dev_reference"]

    # For Pronation
    merged["accel_pronation_selected"] = (merged["frame10_pronation_selected"] - merged["frame2_pronation_selected"]) / 8.0
    merged["accel_pronation_reference"] = (merged["frame10_pronation_reference"] - merged["frame2_pronation_reference"]) / 8.0
    merged["diff_accel_pronation"] = merged["accel_pronation_selected"] - merged["accel_pronation_reference"]

    rows = []
    for _, row in merged.iterrows():
        pitch = row["pitch_type"]
        # Ulnar Deviation row
        rows.append({
            "pitch_type": pitch,
            "rel_u_dev": round(row["rel_u_dev_selected"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"], 1),
            "accel_u_dev": round(row["accel_u_dev_selected"], 1)
        })
        # Ulnar Deviation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Comp",
            "rel_u_dev": round(row["rel_u_dev_selected"] - row["rel_u_dev_reference"], 1),
            "frame2_u_dev": round(row["frame2_u_dev_selected"] - row["frame2_u_dev_reference"], 1),
            "frame4_u_dev": round(row["frame4_u_dev_selected"] - row["frame4_u_dev_reference"], 1),
            "frame6_u_dev": round(row["frame6_u_dev_selected"] - row["frame6_u_dev_reference"], 1),
            "frame8_u_dev": round(row["frame8_u_dev_selected"] - row["frame8_u_dev_reference"], 1),
            "frame10_u_dev": round(row["frame10_u_dev_selected"] - row["frame10_u_dev_reference"], 1),
            "accel_u_dev": round(row["diff_accel_u_dev"], 1)
        })
        # Pronation row (store actual pronation at release in rel_u_dev column)
        rows.append({
            "pitch_type": f"{pitch} Pronation",
            # Instead of "Pronation", place the numeric pronation at release:
            "rel_u_dev": round(row["rel_pronation_selected"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"], 1),
            "accel_u_dev": round(row["accel_pronation_selected"], 1)
        })
        # Pronation Comparison row
        rows.append({
            "pitch_type": f"{pitch} Pronation Comp",
            "rel_u_dev": round(row["rel_pronation_selected"] - row["rel_pronation_reference"], 1),
            "frame2_u_dev": round(row["frame2_pronation_selected"] - row["frame2_pronation_reference"], 1),
            "frame4_u_dev": round(row["frame4_pronation_selected"] - row["frame4_pronation_reference"], 1),
            "frame6_u_dev": round(row["frame6_pronation_selected"] - row["frame6_pronation_reference"], 1),
            "frame8_u_dev": round(row["frame8_pronation_selected"] - row["frame8_pronation_reference"], 1),
            "frame10_u_dev": round(row["frame10_pronation_selected"] - row["frame10_pronation_reference"], 1),
            "accel_u_dev": round(row["diff_accel_pronation"], 1)
        })

    comp_df = pd.DataFrame(rows)
    return comp_df


# ----------------- DASH APP SETUP -----------------
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
app.title = "Pitch Analysis Dashboard"

logo_url = "https://8ctanebaseball.com/wp-content/uploads/2024/02/cropped-8ctaneBaseballLogo-2.png"

# We define global defaults from the last inserted row:
default_participant = LAST_PARTICIPANT
default_date = LAST_DATE

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.Label("Select Participant", style={"color": "white"}),
            dcc.Dropdown(
                id="participant-dropdown",
                options=get_dropdown_options(),
                value=default_participant,  # Use last processed participant
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Test Date", style={"color": "white"}),
            dcc.Dropdown(
                id="date-dropdown",
                # We leave options empty; the callback will populate them
                options=[],
                value=default_date,  # Use last processed date
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Type", style={"color": "white"}),
            dcc.Dropdown(
                id="pitch-type-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Label("Select Pitch Number", style={"color": "white"}),
            dcc.Dropdown(
                id="filename-dropdown",
                style={
                    "backgroundColor": "black",
                    "color": "white",
                    "border": "1px solid #666",
                    "width": "100%",
                }
            )
        ], width=2),

        dbc.Col([
            html.Div([
                html.Img(
                    src=logo_url,
                    style={
                        "position": "relative", 
                        "width": "440px",
                        "height": "auto",
                        "padding": "10px",  
                        "margin-left": "140px",
                        "backgroundColor": "black"  
                    }
                )
            ], style={"text-align": "right"})
        ], width=2)
    ], className="mt-3", align="center"),       

    html.Hr(),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Comparison Table"),
                dbc.CardBody(
                    dash_table.DataTable(
                        id="comparison-table",
                        columns=[{"name": i, "id": i} for i in [
                            "pitch_type", "rel_u_dev",
                            "frame2_u_dev", "frame4_u_dev",
                            "frame6_u_dev", "frame8_u_dev",
                            "frame10_u_dev", "accel_u_dev"
                        ]],
                        data=[],
                        style_table={"overflowX": "auto"},
                        style_cell={
                            "textAlign": "center",
                            "color": "white",
                            "backgroundColor": "black"
                        },
                        style_header={
                            "backgroundColor": "#333333",
                            "color": "white"
                        },
                        page_size=10
                    )
                )
            ]),
            width=12
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Ulnar Deviation Time Series"),
                dbc.CardBody(dcc.Graph(id="ulnar-dev-graph"))
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Acceleration: Transverse & Frontal"),
                dbc.CardBody(dcc.Graph(id="acceleration-graph"))
            ]), width=6
        )
    ], className="mt-3"),

    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Pronation Time Series"),
                dbc.CardBody(dcc.Graph(id="pronation-graph"))
            ]), width=6
        ),
        dbc.Col(
            dbc.Card([
                dbc.CardHeader("Flexion Time Series"),
                dbc.CardBody(dcc.Graph(id="flexion-graph"))
            ]), width=6
        )
    ], className="mt-3")
], fluid=True)


# ----------------- DASH CALLBACKS -----------------
@app.callback(
    Output("date-dropdown", "options"),
    Output("date-dropdown", "value"),
    Input("participant-dropdown", "value")
)
def update_date_dropdown(selected_participant):
    if not selected_participant:
        return [], None
    options = get_date_options(selected_participant)
    # If the default_date is in the new options, keep it, else use the first
    possible_values = [o["value"] for o in options]
    if default_date in possible_values:
        return options, default_date
    return options, (options[0]["value"] if options else None)

@app.callback(
    Output("pitch-type-dropdown", "options"),
    Output("pitch-type-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value")
)
def update_pitch_type_options(selected_participant, selected_date):
    if not selected_participant or not selected_date:
        return [], None
    options = get_pitch_type_options(selected_participant, selected_date)
    pitch_types = [o["value"] for o in options]

    # Default to "Curve" if available, else "All" or the first
    if "Curve" in pitch_types:
        default_value = "Curve"
    else:
        default_value = "All" if "All" in pitch_types else (pitch_types[0] if pitch_types else None)

    return options, default_value

@app.callback(
    Output("filename-dropdown", "options"),
    Output("filename-dropdown", "value"),
    Input("participant-dropdown", "value"),
    Input("date-dropdown", "value"),
    Input("pitch-type-dropdown", "value")
)
def update_filename_options(selected_participant, selected_date, selected_pitch_type):
    if not selected_participant or not selected_date or not selected_pitch_type:
        return [], None
    options = get_filename_options(selected_participant, selected_date, selected_pitch_type)
    value = options[0]["value"] if options else None
    return options, value

@app.callback(
    Output("comparison-table", "data"),
    Output("ulnar-dev-graph", "figure"),
    Output("acceleration-graph", "figure"),
    Output("pronation-graph", "figure"),
    Output("flexion-graph", "figure"),
    [
        Input("participant-dropdown", "value"),
        Input("date-dropdown", "value"),
        Input("pitch-type-dropdown", "value"),
        Input("filename-dropdown", "value")
    ]
)
def update_dashboard(selected_participant, selected_date, selected_pitch_type, selected_filename):
    # 1) Table
    comp_df = get_comparison_table(selected_participant, selected_date, selected_pitch_type, selected_filename)
    table_data = comp_df.to_dict("records")

    # 2) main time-series
    ts_data = get_time_series(selected_participant, selected_date, selected_pitch_type, selected_filename)

    # 3) reference data (only "Curve" for example)
    ref_data = get_curve_reference_on_the_fly()  # or your reference table approach

    # Vertical line at average release
    release_frames = []
    for pt, data_dict in ts_data.items():
        for series in data_dict["ulnar_dev_series"]:
            # release is at index = len(series) - 20
            release_frames.append(len(series) - 20)
    avg_release = int(np.mean(release_frames)) if release_frames else 20

    vline = dict(
        type="line",
        x0=avg_release, x1=avg_release,
        y0=0, y1=1,
        xref="x", yref="paper",
        line=dict(color="#dfb16d", width=2, dash="dash")
    )

    pitch_color_map = {
        "Fastball": "#d79ea5",
        "Curve": "#2c99d4",
        "Slider": "#ff9900",
        "Changeup": "#ffff00"
    }

    # --------------------------------
    # Ulnar Deviation
    # --------------------------------
    ulnar_fig = go.Figure()
    # (a) reference
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    # (b) actual data
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        for series in data_dict["ulnar_dev_series"]:
            ulnar_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt
            ))
    ulnar_fig.update_layout(
        title="Ulnar Deviation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    # --------------------------------
    # Acceleration
    # --------------------------------
    accel_fig = go.Figure()

    def compute_acceleration(angle_series):
        # 2nd difference = approximate acceleration
        # Adjust as needed (frame rate, etc.)
        return np.diff(angle_series, n=2)

    def compute_rms(signal, window_size=5):
        squared = np.square(signal)
        kernel = np.ones(window_size) / window_size
        mean_sq = np.convolve(squared, kernel, mode='same')
        return np.sqrt(mean_sq)

    for pt, data_dict in ts_data.items():
        # Ulnar deviation acceleration
        for series in data_dict["ulnar_dev_series"]:
            accel_u = compute_acceleration(series)
            rms_u = compute_rms(accel_u)
            x_vals = list(range(len(rms_u)))
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_u,
                mode="lines",
                line=dict(color="#bb6a74", width=2),
                name=f"{pt} Ulnar RMS"
            ))

        # Pronation acceleration
        for series in data_dict["pronation"]:
            accel_p = compute_acceleration(series)
            rms_p = compute_rms(accel_p)
            x_vals = list(range(len(rms_p)))
            accel_fig.add_trace(go.Scatter(
                x=x_vals,
                y=rms_p,
                mode="lines",
                line=dict(color="#2c99d4", width=2),
                name=f"{pt} Pronation RMS"
            ))

    accel_fig.update_layout(
        title="Acceleration: Transverse & Frontal",
        xaxis_title="Frame",
        yaxis_title="Accel (°/frame²)",
        shapes=[vline]
    )

    # --------------------------------
    # Pronation
    # --------------------------------
    pronation_fig = go.Figure()
    # (a) reference
    if selected_pitch_type in ref_data:
        for series in ref_data[selected_pitch_type]["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{selected_pitch_type} Reference"
            ))
    # (b) actual data (FIX: use ts_data, not ref_data again!)
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        for series in data_dict["pronation"]:
            pronation_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt
            ))
    pronation_fig.update_layout(
        title="Pronation Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    # --------------------------------
    # Flexion
    # --------------------------------
    flexion_fig = go.Figure()
    # (a) reference
    # No pitch-type filter if your ref_data only has "Curve", 
    # or do the same check as above:
    for pt, data_dict in ref_data.items():
        for series in data_dict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color="rgba(88,108,123,0.5)", width=35),
                name=f"{pt} Reference"
            ))
    # (b) actual data
    for pt, data_dict in ts_data.items():
        color = pitch_color_map.get(pt, "#999999")
        for series in data_dict["flexion"]:
            flexion_fig.add_trace(go.Scatter(
                x=list(range(len(series))),
                y=series,
                mode="lines",
                line=dict(color=color, width=2),
                name=pt
            ))
    flexion_fig.update_layout(
        title="Flexion Time Series",
        xaxis_title="Frame",
        yaxis_title="Angle (°)",
        shapes=[vline]
    )

    return table_data, ulnar_fig, accel_fig, pronation_fig, flexion_fig


if __name__ == '__main__':
    app.run_server(debug=True)


Inserted data for Slider RH 3 (Slider)
Inserted data for Slider RH 2 (Slider)
Inserted data for Slider RH 1 (Slider)
Inserted data for Changeup RH 2 (Changeup)
Inserted data for Changeup RH 1 (Changeup)
Inserted data for Curve RH 6 (Curve)
Inserted data for Curve RH 5 (Curve)
Inserted data for Curve RH 4 (Curve)
Inserted data for Curve RH 3 (Curve)
Inserted data for Curve RH 1 (Curve)
Inserted data for Fastball RH 3 (Fastball)
Inserted data for Slider RH 5 (Slider)
Inserted data for Fastball RH 2 (Fastball)
Inserted data for Slider RH 4 (Slider)
Skipping D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Fastball RH 1.c3d: Required events (Foot Contact, Release) not found in C3D events.
Inserted data for Curve RH 2 (Curve)
Skipping D:/Youth Pitch Design/Data/Bobby Wahl_BW/2025-02-05_\Pitch Design Static RH 1.c3d: No EVENT data in C3D.
All data inserted into pitch_data!


In [62]:
import pdfkit
import tkinter as tk

root = tk.Tk()
root.withdraw()
target_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")

config = pdfkit.configuration(wkhtmltopdf=r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe")

pdfkit.from_url(
    "http://127.0.0.1:8050",
    "C:/SomeFolder/dashboard.pdf",
    configuration=config
)


OSError: No wkhtmltopdf executable found: "C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe"
If this file exists please check that this process can read it or you can pass path to it manually in method call, check README. Otherwise please install wkhtmltopdf - https://github.com/JazzCore/python-pdfkit/wiki/Installing-wkhtmltopdf

In [1]:
# Add to reference table in db

import ezc3d
import os
import numpy as np
import sqlite3
import tkinter as tk
from tkinter import filedialog

# ----------------- SETUP DB -----------------
db_path = "pitch_analysis.sqlite"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS reference_data (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    participant_name TEXT,
    pitch_date TEXT,
    pitch_type TEXT,
    filename TEXT,
    mid_u_dev REAL,
    rel_u_dev REAL,
    frame1_u_dev REAL,
    frame2_u_dev REAL,
    frame3_u_dev REAL,
    frame4_u_dev REAL,
    frame5_u_dev REAL,
    frame6_u_dev REAL,
    frame7_u_dev REAL,
    frame8_u_dev REAL,
    frame9_u_dev REAL,
    frame10_u_dev REAL,
    mid_pronation REAL,
    rel_pronation REAL,
    frame1_pronation REAL,
    frame2_pronation REAL,
    frame3_pronation REAL,
    frame4_pronation REAL,
    frame5_pronation REAL,
    frame6_pronation REAL,
    frame7_pronation REAL,
    frame8_pronation REAL,
    frame9_pronation REAL,
    frame10_pronation REAL
)
""")
conn.commit()
conn.close()

# ----------------- USER INPUT (folder selection) -----------------
root = tk.Tk()
root.withdraw()
selected_folder = filedialog.askdirectory(title="Select Data Folder")
if not selected_folder:
    raise ValueError("No folder was selected.")

# ----------------- LOAD C3D FILE PATHS -----------------
c3d_files = [
    os.path.join(selected_folder, file)
    for file in os.listdir(selected_folder)
    if file.lower().endswith('.c3d')
]
if not c3d_files:
    raise FileNotFoundError("No C3D files found in the selected folder.")

# ----------------- HELPER FUNCTIONS -----------------
def compute_ulnar_deviation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    H = points[:3, marker_indices["Hand"], frame]
    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = E_center - W_center
    radial_vec  = R - U
    hand_vec    = H - W_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    radial_proj = radial_vec - np.dot(radial_vec, forearm_unit)*forearm_unit
    hand_proj   = hand_vec   - np.dot(hand_vec,   forearm_unit)*forearm_unit
    if np.linalg.norm(radial_proj) < 1e-9 or np.linalg.norm(hand_proj) < 1e-9:
        return 0.0
    radial_proj_n = radial_proj / np.linalg.norm(radial_proj)
    hand_proj_n   = hand_proj   / np.linalg.norm(hand_proj)
    cross_val = np.cross(radial_proj_n, hand_proj_n)
    dot_val   = np.dot(radial_proj_n, hand_proj_n)
    angle_rad = np.arctan2(np.linalg.norm(cross_val), dot_val)
    sign = np.sign(np.dot(cross_val, forearm_unit))
    angle_deg = np.degrees(angle_rad) * sign
    return 180 - abs(angle_deg)

def compute_pronation(points, marker_indices, frame):
    R = points[:3, marker_indices["Wrist_Radius"], frame]
    U = points[:3, marker_indices["Wrist_Ulna"], frame]
    E_lat = points[:3, marker_indices["Lateral_Elbow"], frame]
    E_med = points[:3, marker_indices["Medial_Elbow"], frame]
    W_center = (R + U) / 2.0
    E_center = (E_lat + E_med) / 2.0
    forearm_vec = W_center - E_center
    forearm_unit = forearm_vec / (np.linalg.norm(forearm_vec) + 1e-9)
    wrist_vec = R - U
    wrist_proj = wrist_vec - np.dot(wrist_vec, forearm_unit)*forearm_unit
    elbow_vec = E_lat - E_med
    elbow_proj = elbow_vec - np.dot(elbow_vec, forearm_unit)*forearm_unit
    if np.linalg.norm(wrist_proj) < 1e-9 or np.linalg.norm(elbow_proj) < 1e-9:
        return 0.0
    wrist_proj_n = wrist_proj / np.linalg.norm(wrist_proj)
    elbow_proj_n = elbow_proj / np.linalg.norm(elbow_proj)
    cross_val = np.cross(elbow_proj_n, wrist_proj_n)
    dot_val = np.dot(elbow_proj_n, wrist_proj_n)
    angle_rad = np.arccos(np.clip(dot_val, -1.0, 1.0))
    sign = np.sign(np.dot(cross_val, forearm_unit))
    raw_angle = np.degrees(angle_rad) * sign
    return 180 - abs(raw_angle)

def find_local_events(c3d, frame_rate, total_frames):
    if "EVENT" not in c3d["parameters"]:
        raise ValueError("No EVENT data in C3D.")
    event_labels = c3d["parameters"]["EVENT"]["LABELS"]["value"]
    event_times  = c3d["parameters"]["EVENT"]["TIMES"]["value"][1]
    if "Foot Contact" not in event_labels or "Release" not in event_labels:
        raise ValueError("Required events (Foot Contact, Release) not found in C3D events.")
    idx_foot = event_labels.index("Foot Contact")
    idx_release = event_labels.index("Release")
    foot_contact_sec = event_times[idx_foot]
    release_sec = event_times[idx_release]
    foot_contact_global = int(round(foot_contact_sec * frame_rate))
    release_global = int(round(release_sec * frame_rate))
    earliest = min(foot_contact_global, release_global)
    foot_local = foot_contact_global - earliest
    release_local = release_global - earliest
    if foot_local < 0 or foot_local >= total_frames:
        raise ValueError(f"Foot Contact frame {foot_local} out of range.")
    if release_local < 0 or release_local >= total_frames:
        raise ValueError(f"Release frame {release_local} out of range.")
    return foot_local, release_local

def resolve_marker_indices(marker_labels):
    clean_labels = [lab.replace("Right_", "").replace("Left_", "") for lab in marker_labels]
    req = ["Lateral_Elbow", "Medial_Elbow", "Wrist_Radius", "Wrist_Ulna", "Hand"]
    marker_indices = {}
    for mk in req:
        if mk in clean_labels:
            marker_indices[mk] = clean_labels.index(mk)
        else:
            return {}
    return marker_indices

# ------------------------------------------------------------
#  MAIN PROCESSING LOOP: Insert results into reference_data
# ------------------------------------------------------------
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for c3d_file_path in c3d_files:
    normalized_path = os.path.normpath(c3d_file_path)
    path_parts = normalized_path.split(os.sep)
    if len(path_parts) < 3:
        continue

    participant_folder = path_parts[-3]
    participant_name = participant_folder.rsplit("_", 1)[0]
    date_folder = path_parts[-2]
    pitch_date = date_folder.rstrip("_")
    filename_only = path_parts[-1]
    filename_noext = os.path.splitext(filename_only)[0]

    pitch_type = filename_noext.split()[0].capitalize()

    # Read C3D
    try:
        c3d_obj = ezc3d.c3d(c3d_file_path)
    except Exception as e:
        print(f"Error reading {c3d_file_path}: {e}")
        continue

    points = c3d_obj["data"]["points"]
    marker_labels = c3d_obj["parameters"]["POINT"]["LABELS"]["value"]
    frame_rate = c3d_obj["parameters"]["POINT"]["RATE"]["value"][0]
    total_frames = points.shape[2]

    try:
        foot_contact_frame, release_frame = find_local_events(c3d_obj, frame_rate, total_frames)
    except Exception as e:
        print(f"Skipping {c3d_file_path}: {e}")
        continue

    # We want to capture foot_contact..(release+20)
    end_frame = release_frame + 20
    if end_frame >= total_frames:
        print(f"Skipping {c3d_file_path} because end_frame >= total_frames.")
        continue

    markers = resolve_marker_indices(marker_labels)
    if not markers:
        print(f"Skipping {c3d_file_path}: Required markers not found.")
        continue

    # Number of frames in this region
    num_frames = end_frame - foot_contact_frame
    if num_frames < 1:
        print(f"Skipping {c3d_file_path}: no valid frames from foot->release+20.")
        continue

    # We'll define 'mid_frame' in the extended region as halfway from foot_contact to end_frame.
    mid_frame = foot_contact_frame + (num_frames // 2)

    # 11 equally spaced frames from foot_contact to end_frame
    #   => frame0 = foot_contact, frame10 = release+20
    equally_spaced = np.linspace(foot_contact_frame, end_frame, num=11, dtype=int)

    # ------------------ Ulnar Deviation ------------------
    mid_u_dev = compute_ulnar_deviation(points, markers, mid_frame)
    rel_u_dev = compute_ulnar_deviation(points, markers, release_frame)

    frame_u_dev = []
    # frames 1..10 correspond to foot->end spaced points,
    # i.e. [ foot_contact + X*(end_frame-foot_contact)/10 ]
    # so frame1..frame10 are the interior (1..10).
    for i in range(1, 11):
        f_idx = equally_spaced[i]
        angle = compute_ulnar_deviation(points, markers, f_idx)
        frame_u_dev.append(angle)

    # ------------------ Pronation ------------------
    mid_pronation = compute_pronation(points, markers, mid_frame)
    rel_pronation = compute_pronation(points, markers, release_frame)

    frame_pronation = []
    for i in range(1, 11):
        f_idx = equally_spaced[i]
        angle = compute_pronation(points, markers, f_idx)
        frame_pronation.append(angle)

    # -------------------------------------------------
    # Insert into reference_data
    # -------------------------------------------------
    insert_sql = """
    INSERT INTO reference_data (
        participant_name, pitch_date, pitch_type, filename,
        mid_u_dev, rel_u_dev,
        frame1_u_dev, frame2_u_dev, frame3_u_dev, frame4_u_dev, frame5_u_dev,
        frame6_u_dev, frame7_u_dev, frame8_u_dev, frame9_u_dev, frame10_u_dev,
        mid_pronation, rel_pronation,
        frame1_pronation, frame2_pronation, frame3_pronation, frame4_pronation, frame5_pronation,
        frame6_pronation, frame7_pronation, frame8_pronation, frame9_pronation, frame10_pronation
    )
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
    """

    data_tuple = (
        participant_name,
        pitch_date,
        pitch_type,
        filename_noext,

        float(mid_u_dev),
        float(rel_u_dev),
        float(frame_u_dev[0]),
        float(frame_u_dev[1]),
        float(frame_u_dev[2]),
        float(frame_u_dev[3]),
        float(frame_u_dev[4]),
        float(frame_u_dev[5]),
        float(frame_u_dev[6]),
        float(frame_u_dev[7]),
        float(frame_u_dev[8]),
        float(frame_u_dev[9]),

        float(mid_pronation),
        float(rel_pronation),
        float(frame_pronation[0]),
        float(frame_pronation[1]),
        float(frame_pronation[2]),
        float(frame_pronation[3]),
        float(frame_pronation[4]),
        float(frame_pronation[5]),
        float(frame_pronation[6]),
        float(frame_pronation[7]),
        float(frame_pronation[8]),
        float(frame_pronation[9])
    )

    cursor.execute(insert_sql, data_tuple)
    print(f"Inserted data for {filename_noext} ({pitch_type})")

conn.commit()
conn.close()
print("All data inserted into reference_data!")


Inserted data for Slider RH 3 (Slider)
Inserted data for Slider RH 2 (Slider)
Inserted data for Slider RH 1 (Slider)
Inserted data for Changeup RH 2 (Changeup)
Inserted data for Curve RH 3 (Curve)
Inserted data for Curve RH 2 (Curve)
Inserted data for Fastball RH 3 (Fastball)
Inserted data for Fastball RH 2 (Fastball)
Skipping D:/Youth Pitch Design/Data/Reference Data_RD/2025-02-21_\Fastball RH 1.c3d: Required events (Foot Contact, Release) not found in C3D events.
Inserted data for Changeup RH 1 (Changeup)
Skipping D:/Youth Pitch Design/Data/Reference Data_RD/2025-02-21_\Pitch Design Static RH 2.c3d: No EVENT data in C3D.
Inserted data for Curve RH 1 (Curve)
All data inserted into reference_data!


In [ ]:
######################################################
# All below this is drafts
######################################################